# Generate picks for today's games
This code will likely be used in the web application

In [21]:
import pickle
import pandas as pd
import requests
from datetime import datetime
from zoneinfo import ZoneInfo

In [22]:
nba_games_model = pickle.load(open('data/2024_random_forest_model.pkl', 'rb'))
nba_games = pd.read_csv('data/04_improved_model_with_moneylines.csv', dtype={'GAME_ID': str}, parse_dates=['GAME_DATE_EST'], date_format='%Y-%m-%d')
nba_games = nba_games.sort_values(by='GAME_DATE_EST')

In [ ]:
nba_games.info()

In [24]:

HEADERS = {
  "Referer": "stats.nba.com",
  "Content-Type": "application/json",
  "Accept": "*/*",
  "Accept-Encoding": "gzip, deflate, br",
  "Connection": "keep-alive",
  "Host": "stats.nba.com",
  "Origin": "https://stats.nba.com",
  "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0",
}
eastern = ZoneInfo('US/Eastern')
current_date = datetime.now(eastern).strftime('%m/%d/%Y')
url = f"https://stats.nba.com/stats/scoreboardV2?DayOffset=0&LeagueID=00&gameDate={current_date}"

res = requests.get(url, headers=HEADERS).json()

In [25]:
def calc_last_n_win_pct(team_id, n):
    _game = nba_games[(nba_games['HOME_TEAM_ID'] == team_id) | (nba_games['AWAY_TEAM_ID'] == team_id)]
    _game['IS_HOME'] = _game['HOME_TEAM_ID'] == team_id
    _game['WIN_PRCT'] = _game['IS_HOME'] == _game['HOME_TEAM_WINS']
    return _game["WIN_PRCT"].tail(n).mean()


In [ ]:
nba_games_model.feature_names_in_

In [ ]:
results = res['resultSets']
game_headers = results[0]
eastern_standings = results[4]
western_standings = results[5]

games = []
for game in game_headers.get('rowSet'):
  game_id = game[2]
  home_team_id = game[6]
  away_team_id = game[7]
  season = game[8]

  if home_team_id in [r[0] for r in eastern_standings.get("rowSet")]:
      home_team_rank = next((team for team in eastern_standings.get("rowSet") if team[0] == home_team_id))
  else:
      home_team_rank = next((team for team in western_standings.get("rowSet") if team[0] == home_team_id))

  if away_team_id in [r[0] for r in eastern_standings.get("rowSet")]:
      away_team_rank = next((team for team in eastern_standings.get("rowSet") if team[0] == away_team_id))
  else:
      away_team_rank = next((team for team in western_standings.get("rowSet") if team[0] == away_team_id))

  home_win_pct = home_team_rank[9]
  home_home_wins, home_home_losses = home_team_rank[10].split("-")
  home_home_win_pct = int(home_home_wins) / (int(home_home_wins) + int(home_home_losses)) if int(home_home_wins) + int(home_home_losses) > 0 else 0

  away_win_pct = away_team_rank[9]
  away_away_wins, away_away_losses = away_team_rank[11].split("-")
  away_away_win_pct = int(away_away_wins) / (int(away_away_wins) + int(away_away_losses)) if int(away_away_wins) + int(away_away_losses) > 0 else 0

  game_date = datetime.strptime(game[0], "%Y-%m-%dT%H:%M:%S")
  yesterday = game_date - pd.Timedelta(days=1)

  home_yesterday_game_count = len(nba_games[(nba_games['GAME_DATE_EST'] == yesterday) & ((nba_games['HOME_TEAM_ID'] == home_team_id) | (nba_games["AWAY_TEAM_ID"] == home_team_id))])
  home_b2b = home_yesterday_game_count > 0

  away_yesterday_game_count = len(nba_games[(nba_games['GAME_DATE_EST'] == yesterday) & ((nba_games['HOME_TEAM_ID'] == away_team_id) | (nba_games["AWAY_TEAM_ID"] == away_team_id))])
  away_b2b = away_yesterday_game_count > 0

  home_last_10_win_pct = calc_last_n_win_pct(home_team_id, 10)
  away_last_10_win_pct = calc_last_n_win_pct(away_team_id, 10)


  games.append({
      "GAME_DATETIME": int(game_date.timestamp()) * 10**9,
      "HOME_TEAM_ID": home_team_id,
      "AWAY_TEAM_ID": away_team_id,
      "HOME_WIN_PCT": home_win_pct,
      "HOME_HOME_WIN_PCT": home_home_win_pct,
      "AWAY_WIN_PCT": away_win_pct,
      "AWAY_AWAY_WIN_PCT": away_away_win_pct,
      "HOME_TEAM_B2B": home_b2b,
      "AWAY_TEAM_B2B": away_b2b,
      "HOME_LAST_10_WIN_PCT": home_last_10_win_pct,
      "AWAY_LAST_10_WIN_PCT": away_last_10_win_pct
  })

In [ ]:
games

In [29]:
games_df = pd.DataFrame(games)
games_df["PREDICTION"] = nba_games_model.predict(games_df)

In [30]:
teams = pd.read_csv('data/raw/nba_teams.csv')

In [ ]:
games_df = games_df.merge(teams[["TEAM_ID", "ABBREVIATION", "NICKNAME"]], left_on='HOME_TEAM_ID', right_on='TEAM_ID')
games_df = games_df.merge(teams[["TEAM_ID", "ABBREVIATION", "NICKNAME"]], left_on='AWAY_TEAM_ID', right_on='TEAM_ID', suffixes=('_HOME', '_AWAY'))
games_df.drop(columns=["TEAM_ID_HOME", "TEAM_ID_AWAY"], inplace=True)
games_df

In [32]:
def prettify_winner(row):
    if row["PREDICTION"] == 1:
        return f"{row['ABBREVIATION_HOME']} {row['NICKNAME_HOME']}"
    else:
        return f"{row['ABBREVIATION_AWAY']} {row['NICKNAME_AWAY']}"

games_df["WINNER"] = games_df.apply(prettify_winner, axis=1)

In [33]:
games_df["GAME_DATE_EST"] = pd.to_datetime(games_df["GAME_DATETIME"], unit='ns').dt.strftime('%Y-%m-%d')

In [ ]:
games_df[["GAME_DATE_EST", "ABBREVIATION_HOME", "NICKNAME_HOME", "ABBREVIATION_AWAY", "NICKNAME_AWAY", "WINNER"]].to_json(orient='records')